Now that we have looked at accessing Dataframes, let look at using them!

In [38]:
import pandas as pd
import numpy as np
import time 

df = pd.read_csv('congress-terms.csv', on_bad_lines='skip')


In [39]:
%%timeit -r 10 -n 10
# Iterrows
total=[]
for index, row in df.iterrows():
    total.append(row['firstname']+row['lastname'])

594 ms ± 7.23 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [40]:
%%timeit -r 10 -n 10
# For loop with .iloc
total = []
for index in range(len(df)):
        total.append(df.iloc[index][3]+df.iloc[index][5])

2.61 s ± 58.7 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [41]:
%%timeit -r 10 -n 10
# For loop with .loc
total = []
for index in range(len(df)):
        total.append(df.loc[index]['firstname']+df.loc[index]['lastname'])

3.06 s ± 108 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [42]:
%%timeit -r 10 -n 10
# Apply
df.apply(lambda row: row['firstname'] + row['lastname'], axis=1).to_list()


196 ms ± 2.38 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [43]:
%%timeit -r 10 -n 10
total = []
for row in df.itertuples():
    total.append(row.firstname + row.lastname)

23.8 ms ± 618 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [44]:
%%timeit -r 10 -n 100
# List comprehension
[first + last for first, last in zip(df['firstname'], df['lastname'])]

4.02 ms ± 103 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [45]:
%%timeit -r 10 -n 100
(df['firstname'] + df['lastname']).to_list()

1.24 ms ± 31.7 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [46]:
%%timeit -r 10 -n 100
(df['firstname'].to_numpy() + df['lastname'].to_numpy()).tolist()

1.15 ms ± 43.3 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


Vectorization!

Numpy